#### Imports

La idea de esta variante, es predecir solamente los productos que tengan mas de 12 meses de datos (usando los ultimos 12 solamente), y aquellos con menos datos, predecir la media

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from skopt import BayesSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.metrics import MeanSquaredError
from keras.regularizers import l2
from keras.layers import BatchNormalization
from skopt.space import Categorical
from sklearn.metrics import mean_squared_error
import json

In [ ]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [ ]:
final_dataset.head()

In [ ]:
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter', 'cat1', 'cat2', 'cat3', 'close_quarter', 'y']
non_scalable_columns = ['cat1', 'cat2', 'cat3', 'plan_precios_cuidados', 'close_quarter']
n_features = len(columns)

In [ ]:
categorical_columns = ['cat1', 'cat2', 'cat3']

# Transformar las columnas categóricas en numéricas
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    final_dataset[col] = le.fit_transform(final_dataset[col])
    label_encoders[col] = le

In [ ]:
grouped = final_dataset.groupby(['product_id', 'cat1', 'cat2', 'cat3'])
display(grouped)

In [ ]:
scalers = {col: {} for col in columns}
scaled_data_list = []

for (product_id, _, _, _ ), group in grouped:
    scaled_group = group.copy()
    for col in columns:
        if col not in non_scalable_columns:
            scaler = StandardScaler()
            scaled_group[col] = scaler.fit_transform(group[[col]])
            scalers[col][product_id] = scaler
    scaled_data_list.append(scaled_group)

# Combinar todos los datos escalados en un solo DataFrame
scaled_data = pd.concat(scaled_data_list)

In [ ]:
display(scaled_data[columns])

In [ ]:
# scaled_data[scaled_data['cat1'] != 0]

#### Funcion para preparar los datos y crear el modelo

El objetivo es predecir 2 dias en el futuro, por lo que la idea es re-armar el dataset. Donde el valor de X sera el conjunto de datos hasta N-2 e Y va a ser N (siendo N la cantidad de ventas para ese producto)

In [103]:
def prepare_data(data,validation=False):
    X, y = [], []
    unique_product_ids = data['product_id'].unique()
    predicciones = {}

    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        if len(product_values) < 14:
            if validation:
                predicciones[product_id] = np.mean(product_values)
            continue
        
        
        if validation:
            X.append(product_values[-12:])
            y.append(product_id)
        else:
            # Todas las observaciones menos las últimas dos. Como despues necesito agregar esos 2 registros
            # para predecir el future, le agrego un pad left extra de 2 para no romper el shape de la red
            X.append(product_values[-14:-2])
            y.append(product_values[-1, -1])      # Última observación
    
    if validation:
        return np.array(X), np.array(y), predicciones
    else:
        return np.array(X), np.array(y), predicciones


Definimos la funcion para crear el modelo LSTM, sobre este se ejecutara la optimizacion bayesiana

In [ ]:
n_steps = 2  # número de pasos de tiempo
epochs = 100
batch_size = 32
predictions = []

In [ ]:
data = scaled_data[columns].values
X, y, _ = prepare_data(scaled_data)

In [85]:
display(X.shape)
display(y.shape)

(656, 12, 9)

(656,)

#### Revisamos que la funcion genere registros coherentes

In [ ]:
# Muestro todos los registros del ultimo producto
unique_product_ids = scaled_data['product_id'].unique()
product_data = scaled_data[scaled_data['product_id'] == unique_product_ids[0]] #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft
product_values = product_data[columns].values
display(product_values)

In [ ]:
display(X[0]) #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft

In [ ]:
display(y[0]) #<= cambiar el indice entre 0 y -1

In [ ]:
display(len(X[0]))

#### Entrenamiento del modelo

In [ ]:
from keras.regularizers import l2

max_seq_length = 12 #cantidad de mese a usar por producto

def create_model(units, learning_rate, optimizer='adam', dropout=0.1, activation="tanh", depth=2, l2_penalty=1e-6):
    display(f"Creating model with parameters: units={units}, learning_rate={learning_rate}, "
                f"optimizer={optimizer}, dropout={dropout}, activation={activation}, "
                f"depth={depth}, l2_penalty={l2_penalty}")
    
    model = Sequential()
    
    for i in range(depth):
        if i == 0:
            model.add(LSTM(units=units, return_sequences=True, input_shape=(max_seq_length, n_features), activation=activation, kernel_regularizer=l2(l2_penalty)))
            model.add(BatchNormalization())
        else:
            model.add(LSTM(units=units, return_sequences=True, activation=activation, kernel_regularizer=l2(l2_penalty)))
            model.add(Dropout(dropout))
            model.add(BatchNormalization())
    
    model.add(LSTM(units=units, return_sequences=False, activation=activation, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    
    model.add(Dense(1))
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    
    # Usamos el error cuadratico medio, porque penaliza errores grandes. Como tenemos ciertos
    # productos "estrella", para los cuales hay mas ventas, y tener un error en lo mismos
    # puede implicar una gran diferencia de las estimaciones, decidimos probar con esta funcion de perdida.
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[MeanSquaredError()])
    return model

#### Armado de los modelos

In [ ]:
param_space = {
    'depth': [1, 2, 3, 4],#, 5, 6],
    'activation': ['relu', 'tanh'], #'selu', 'swish'
    'units': [64, 128, 256, 512],
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': Categorical(categories=(0.0001, 0.001), prior=None),#0.01
    'epochs': [10, 20, 30, 50, 100],
    'batch_size': Categorical([32, 128], prior=None),#[32, 64, 128],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'l2_penalty': [1e-6, 1e-2, 1e-1]
}

In [ ]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=5) mean_squared_error
early_stopping = EarlyStopping(monitor='mean_squared_error', patience=5)
callbacks = [early_stopping]

model = KerasRegressor(
    build_fn=create_model,
    verbose=1,
    units=64,
    learning_rate=0.01,
    dropout=0.1,
    activation="tanh",
    depth=1,
    l2_penalty=1e-6,
    callbacks = callbacks
)

# Como ahora cada observacion, representa la serie de un producto, no hace falta usar 
# un cv especifico de time series, con el cv normal deberia ser suficiente.
# cv = TimeSeriesSplit(n_splits=5).split(X)

bayes_search = BayesSearchCV(
    model,
    param_space,
    # n_iter=50,
    cv=10,
    verbose=1,
    # n_jobs=-1,
    scoring='neg_mean_squared_error',
)

In [ ]:
bayes_search.fit(X, y, verbose=1)

#### Guardo los mejores hiperparametros

In [79]:
best_params = bayes_search.cv_results_['params'][bayes_search.best_index_]
display(best_params)

best_model = create_model(
    units=best_params['units'],
    learning_rate=best_params['learning_rate'],
    optimizer=best_params['optimizer'],
    dropout=best_params['dropout'],
    activation=best_params['activation'],
    depth=best_params['depth'],
    l2_penalty=best_params['l2_penalty'],
)
best_model.fit(X, y, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)

display(best_params)
display(best_model)

OrderedDict([('activation', 'relu'),
             ('batch_size', 32),
             ('depth', 1),
             ('dropout', 0.2),
             ('epochs', 30),
             ('l2_penalty', 1e-06),
             ('learning_rate', 0.001),
             ('optimizer', 'sgd'),
             ('units', 128)])

'Creating model with parameters: units=128, learning_rate=0.001, optimizer=sgd, dropout=0.2, activation=relu, depth=1, l2_penalty=1e-06'

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - loss: 0.9706 - mean_squared_error: 0.9704
Epoch 2/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.8031 - mean_squared_error: 0.8029
Epoch 3/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.7816 - mean_squared_error: 0.7814
Epoch 4/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.7327 - mean_squared_error: 0.7324
Epoch 5/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.6785 - mean_squared_error: 0.6782
Epoch 6/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.6841 - mean_squared_error: 0.6838
Epoch 7/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.6973 - mean_squared_error: 0.6970
Epoch 8/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.7076 - mean_squared_error: 0.7074
Epoch 9/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.6311 - mean_squared_error: 0.6308
Epoch 10/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.6159 - mean_squared_error: 0.6157
Epoch 11/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/st

OrderedDict([('activation', 'relu'),
             ('batch_size', 32),
             ('depth', 1),
             ('dropout', 0.2),
             ('epochs', 30),
             ('l2_penalty', 1e-06),
             ('learning_rate', 0.001),
             ('optimizer', 'sgd'),
             ('units', 128)])

<Sequential name=sequential_502, built=True>

In [80]:
y_pred = best_model.predict(X)
mean_squared_error(y, y_pred)

21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step


0.534394194642544

In [81]:
# Obtener los mejores hiperparámetros
best_params = bayes_search.cv_results_['params'][bayes_search.best_index_]

# Guardar los mejores hiperparámetros en un archivo JSON
with open('best_params.json', 'w') as file:
    json.dump(best_params, file)

print('Los mejores hiperparámetros han sido guardados en best_params.json.')


Los mejores hiperparámetros han sido guardados en best_params.json.


In [ ]:
with open('best_params.json', 'r') as file:
    best_params = json.load(file)

# Reconstruir el modelo con los mejores hiperparámetros
mejor_modelo = create_model(units=best_params['units'], learning_rate=best_params['learning_rate'], optimizer=best_params['optimizer'], dropout=best_params['dropout'], activation=best_params['activation'], depth=best_params['depth'], l2_penalty=best_params['l2_penalty'])
mejor_modelo.fit(X, y, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)
# Verificar la configuración del nuevo modelo
print("Configuración del nuevo modelo:")
print(mejor_modelo.summary())

#### Hago las predicciones

In [ ]:
# best_params = BayesSearchCV.best_params_

# best_units = best_params['build_fn__units']
# best_dropout_rate = best_params['build_fn__dropout_rate']
# best_optimizer = best_params['build_fn__optimizer']

# best_model = create_model(units=best_units, dropout_rate=best_dropout_rate, optimizer=best_optimizer)
# best_model.fit(X, y)

In [104]:
X_f, product_ids, predictions = prepare_data(scaled_data, validation=True)

In [105]:
scaled_predictions = mejor_modelo.predict(X_f)
display(scaled_predictions)

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


array([[-0.35179213],
       [-0.2227672 ],
       [-1.0607201 ],
       [-0.96052676],
       [-0.8090567 ],
       [-1.0285374 ],
       [-1.0385243 ],
       [-0.92248243],
       [-0.21984027],
       [-0.680777  ],
       [-0.6719221 ],
       [-1.0366627 ],
       [-0.77319735],
       [-0.9706285 ],
       [-0.63779473],
       [-0.84340256],
       [-0.85607857],
       [-1.0615926 ],
       [-1.1376735 ],
       [-0.698301  ],
       [-0.74939156],
       [-0.57719004],
       [-1.022449  ],
       [-0.7247625 ],
       [-0.7168679 ],
       [-0.5362008 ],
       [-0.79473823],
       [-0.7446399 ],
       [-1.0455142 ],
       [-0.9270563 ],
       [-0.73228973],
       [-1.3063878 ],
       [-0.7479686 ],
       [-0.8976931 ],
       [-0.66842604],
       [-0.68114775],
       [-0.77996063],
       [-0.31360522],
       [-0.78650814],
       [-0.7304057 ],
       [-0.65408367],
       [-1.0531667 ],
       [-0.83315915],
       [-0.91870177],
       [-0.72341293],
       [-0

In [106]:
scaled_predictions_1d =  scaled_predictions.reshape(-1)
display(scaled_predictions_1d)

array([-0.35179213, -0.2227672 , -1.0607201 , -0.96052676, -0.8090567 ,
       -1.0285374 , -1.0385243 , -0.92248243, -0.21984027, -0.680777  ,
       -0.6719221 , -1.0366627 , -0.77319735, -0.9706285 , -0.63779473,
       -0.84340256, -0.85607857, -1.0615926 , -1.1376735 , -0.698301  ,
       -0.74939156, -0.57719004, -1.022449  , -0.7247625 , -0.7168679 ,
       -0.5362008 , -0.79473823, -0.7446399 , -1.0455142 , -0.9270563 ,
       -0.73228973, -1.3063878 , -0.7479686 , -0.8976931 , -0.66842604,
       -0.68114775, -0.77996063, -0.31360522, -0.78650814, -0.7304057 ,
       -0.65408367, -1.0531667 , -0.83315915, -0.91870177, -0.72341293,
       -0.8913319 , -1.0244622 , -0.703538  , -0.6758891 , -0.83636814,
       -0.7162055 , -0.8179849 , -0.4935185 , -0.612861  , -0.62354654,
       -0.8598087 , -0.48593384, -0.7391318 , -0.658394  , -0.7206544 ,
       -0.8274422 , -0.6251976 , -0.6586997 , -0.59166414, -0.9253865 ,
       -0.10145919, -0.9417461 , -0.67686576, -0.7177919 , -0.94

In [107]:
predictions_df = pd.DataFrame({
    'product_id': product_ids,
    'predicted_y': scaled_predictions_1d
})

# Iterar sobre cada fila del DataFrame predictions_df
for index, row in predictions_df.iterrows():
    # Obtener el ID del producto y la predicción escalada para este producto
    product_id = row['product_id']
    scaled_prediction = row['predicted_y']
    
    # Obtener el escalador correspondiente a 'predicted_y' para este producto
    scaler = scalers['y'][(product_id)]
    
    # Aplicar la inversa del escalador a la predicción 'predicted_y' para este producto
    inverse_scaled_prediction = scaler.inverse_transform([[scaled_prediction]])[0][0]
    
    # Reemplazar la predicción escalada con la predicción invertida en el DataFrame final
    predictions_df.at[index, 'predicted_y'] = inverse_scaled_prediction

/var/folders/ls/f33jscqd6sb6bd857yb2j2s00000gp/T/ipykernel_42682/3047987798.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1294.9260945789943' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  predictions_df.at[index, 'predicted_y'] = inverse_scaled_prediction


In [108]:
prediction_list = []
for product_id, pronostico in predictions.items():
    prediction_list.append({'product_id': product_id, 'predicted_y': pronostico})

# Convert the list of dictionaries to a DataFrame
predictions_df = pd.concat([predictions_df, pd.DataFrame(prediction_list)], ignore_index=True)

# Save the final DataFrame to a CSV file
predictions_df.to_csv('predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.')
display(predictions_df.shape)

Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.


(780, 2)